In [67]:
# import requests

# publ_webpage = requests.get('https://scs.ifmo.ru/ru/listpublication/show_all_publications.htm') # Получаем страницу со списком публикаций

with open('publ_webpage.html') as file:
    publ_webpage = file.read()

In [68]:
from bs4 import BeautifulSoup as BS

bs = BS(publ_webpage, 'html')

all_publs = bs.main.find(
    "div", {"id": "list", "class": "list"}
).findAll(
    "div", {"class": "group"}
) # Получаем список публикаций, сгруппированных по годам
years = len(all_publs) # Количество лет

In [76]:
all_publs_dict: dict = {}
authors_dict: dict = {}
total_num_of_publs = 0
for i in range(years):
    publs_year = publ_year_list[i].findAll("li") # Получаем список публикаций за год
    num_of_publs = len(publs_year) # Получаем количество публикаций за год
    total_num_of_publs += num_of_publs # Ведем подсчет общего количества публикаций за все время
    year = all_publs[i].find("div", {"class": "label"}).get_text(strip=True).split(' ')[0] # Получаем год
    all_publs_dict[year] = [] # Создаем пустой список для заполнения публикациями за год
    for j in range(num_of_publs): 
        try:
            journal = publs_year[j].find("p").get_text(strip=True).split("//")[1] # Если можно получить журнал, то получаем
        except Exception as e:
            # Если журнал получить нельзя то отрабатываем Exception
            # print("ERROR: " + str(year) + ' ' + str(j) + ' ' + publs_year[j].find("p").get_text(strip=True))
            pass
        authors = publs_year[j].find("h4").get_text(strip=True).split(',')
        all_publs_dict[year].append( # Заполняем словарь
            {
                "authors": authors,
                "publ_name": publs_year[j].find("p").get_text(strip=True).split("//")[0],
                "journal": journal
            }
        )
        # Подсчет авторов
        for author in authors:
            if author in authors_dict.keys():
                authors_dict[author] += 1
            else:
                authors_dict[author] = 1


In [77]:
import pandas as pd
prepare_authors_dataframe = {"author": list(authors_dict.keys()), "value": authors_dict.values()}
df = pd.DataFrame(prepare_authors_dataframe)
df

,author,value
0,Mileshin A.,1
1,Tsopa E.,6
2,Klimenkov S.,6
3,Slapoguzov A.,2
4,Kharitonova A.,1
...,...,...
2053,Максимов А.С.,1
2054,Рычков В.А.,1
2055,Спиваковский А.М.,1
2056,Кудинов А.Т.,1


In [79]:
df.sort_values(by="value")

,author,value
0,Mileshin A.,1
1294,Уткин Л.В.,1
1293,Гуров С.В.,1
1292,Колчин М.А.,1
1291,Хайдарова Р.Р.,1
...,...,...
403,Меженин А.В.,43
639,Щеглов А.Ю.,44
770,Давыдов А.Е.,45
548,Локалов В.А.,48
